In [1]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [6]:
!/usr/local/bin/python3 -m pip install tweepy pandas

  Using cached tweepy-4.15.0-py3-none-any.whl.metadata (4.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
Using cached tweepy-4.15.0-py3-none-any.whl (99 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 10.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [8]:
import tweepy
print("tweepy imported successfully")

tweepy imported successfully


In [12]:
mv ~/Downloads/bitcoin-tweets.zip /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/

mv: rename /Users/gaoyue/Downloads/bitcoin-tweets.zip to /Users/gaoyue/w7/bitcoin_price_prediction_2023106041_YUE-GAO/data/bitcoin-tweets.zip: No such file or directory


In [1]:
import csv

input_file = "../data/bitcoin_tweets_kaggle.csv"
output_file = "../data/bitcoin_tweets_kaggle_cleaned.csv"

# 打开文件，逐行读取
with open(input_file, 'r', encoding='latin1') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    reader = csv.reader(infile, lineterminator='\n')
    writer = csv.writer(outfile, lineterminator='\n')
    
    # 写入表头
    header = next(reader)  # 读取第一行（表头）
    writer.writerow(["date", "text"])  # 只保留 date 和 text 列
    
    # 找到 date 和 text 列的索引
    date_idx = header.index("date")
    text_idx = header.index("text")
    
    # 逐行处理
    count = 0
    for row in reader:
        try:
            # 只提取 date 和 text 列
            writer.writerow([row[date_idx], row[text_idx]])
            count += 1
            if count >= 1000:  # 只取 1000 条
                break
        except (IndexError, UnicodeDecodeError):
            # 跳过无法解析的行
            continue

print(f"Processed {count} rows and saved to {output_file}")

Processed 1000 rows and saved to ../data/bitcoin_tweets_kaggle_cleaned.csv


In [2]:
import pandas as pd

# 加载清理后的文件
kaggle_tweets = pd.read_csv("../data/bitcoin_tweets_kaggle_cleaned.csv")
kaggle_tweets.rename(columns={"date": "timestamp"}, inplace=True)
kaggle_tweets["language"] = "en"  # 添加语言列
tweets = kaggle_tweets.head(1000)  # 确保只取 1000 条

# 保存到 CSV
tweets.to_csv("../data/tweets.csv", index=False)
print(f"Collected {len(tweets)} tweets from Kaggle dataset and saved to data/tweets.csv")

Collected 1000 tweets from Kaggle dataset and saved to data/tweets.csv
